In [21]:
from src.data import DataLoaders
from src import utils
import pathlib
import pandas as pd
#from src.models.ModelClasses import FireRiskModels, SmokeAlarmModels
from matplotlib import pyplot as plt
from pathlib import Path
import geopandas as geo
import folium

import numpy as np

import fiona

import pyproj

In [22]:
out = pd.read_csv(r'.\Data\Model Outputs\combinedModels.csv')
out['GEOID'] = out['GEOID'].str[2:]
out

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,fire_propensity_risk_score,fire_severity_risk_score,recorded_home_visits,detectors_working_prc,detectors_working_CI,geography,"Population Density (per square mile), 2010",detectors_predicted,smoke_detector_risk_score,CombinedRiskScore
0,010010201001,1.342282,2.684564,NaN,NaN,NaN,NaN,NaN,1.342282,1.342282,0.622548,0.506302,0.0,31.38,1.089020,state,504.8,7.960243,0.920398,291.99152
1,010010201002,1.581028,0.790514,NaN,1.581028,0.790514,NaN,1.581028,0.790514,NaN,0.966579,0.824059,0.0,31.38,1.089020,state,504.8,19.468388,0.805316,280.48340
2,010010202001,1.047120,NaN,3.141361,NaN,1.047120,1.047120,1.047120,1.047120,NaN,0.740453,0.082554,0.0,31.38,1.089020,state,1682.5,14.917318,0.850827,285.03445
3,010010202002,2.832861,2.832861,2.832861,0.944287,0.944287,NaN,0.944287,NaN,3.777148,0.853479,0.805192,0.0,31.38,1.089020,state,1682.5,5.594139,0.944059,294.35764
4,010010203001,0.846024,1.269036,0.846024,0.423012,1.269036,0.423012,0.423012,0.846024,NaN,0.994841,0.996304,0.0,31.38,1.089020,state,1633.1,13.131232,0.868688,286.82053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213139,560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.372684,NaN,0.652114,0.248441,0.0,38.90,3.199404,state,1.4,58.149483,0.418505,241.80229
213140,560459511002,0.632111,NaN,NaN,NaN,NaN,NaN,0.632111,NaN,NaN,0.592363,0.505123,0.0,38.90,3.199404,state,1.4,59.694490,0.403055,240.25730
213141,560459513001,0.868810,NaN,0.868810,NaN,0.868810,0.868810,NaN,NaN,1.737619,0.918188,0.485757,3.0,38.90,3.199404,state,91.0,66.851970,0.331480,233.09980
213142,560459513002,NaN,NaN,0.970874,NaN,NaN,0.970874,NaN,NaN,NaN,0.643826,0.615241,3.0,38.90,3.199404,state,91.0,64.308490,0.356915,235.64330


In [23]:
ACS =  DataLoaders.ACSData(2016,'block_group')
ACS = ACS.data

ACS_cols_to_use =[ 'did_not_work_past_12_mo',
               'house_pct_live_alone',
               'pct_alt_heat',
               'house_yr_pct_before_1960',
               'inc_pct_poverty',
               'race_pct_black_or_amind']

ACS = ACS[ACS_cols_to_use]

C:\Users\cle9a\Documents\rcp2_2_18_2022\src\data\DataLoaders.py:72: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ACS.drop('geoid','columns',inplace =True)
C:\Users\cle9a\Documents\rcp2_2_18_2022\src\data\DataLoaders.py:83: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ACS.drop('inc_pcincome','columns',inplace= True)
C:\Users\cle9a\Documents\rcp2_2_18_2022\src\data\DataLoaders.py:92: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ACS.drop(col,'columns', inplace = True)
C:\Users\cle9a\Documents\rcp2_2_18_2022\src\data\DataLoaders.py:92: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ACS.drop(col,'columns', inplace = True)
C:\Users\cle9a\Docume

In [24]:
out = out.merge(ACS, how = 'left', right_index = True, left_on = 'GEOID')
out

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,...,"Population Density (per square mile), 2010",detectors_predicted,smoke_detector_risk_score,CombinedRiskScore,did_not_work_past_12_mo,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind
0,010010201001,1.342282,2.684564,NaN,NaN,NaN,NaN,NaN,1.342282,1.342282,...,504.8,7.960243,0.920398,291.99152,0.355019,0.221831,0.017606,0.221831,0.191946,0.214765
1,010010201002,1.581028,0.790514,NaN,1.581028,0.790514,NaN,1.581028,0.790514,NaN,...,504.8,19.468388,0.805316,280.48340,0.309021,0.127193,0.032895,0.079229,0.044269,0.082213
2,010010202001,1.047120,NaN,3.141361,NaN,1.047120,1.047120,1.047120,1.047120,NaN,...,1682.5,14.917318,0.850827,285.03445,0.509589,0.344560,0.000000,0.135678,0.382199,0.591667
3,010010202002,2.832861,2.832861,2.832861,0.944287,0.944287,NaN,0.944287,NaN,3.777148,...,1682.5,5.594139,0.944059,294.35764,0.309886,0.338496,0.019912,0.235772,0.111426,0.461974
4,010010203001,0.846024,1.269036,0.846024,0.423012,1.269036,0.423012,0.423012,0.846024,NaN,...,1633.1,13.131232,0.868688,286.82053,0.294054,0.276699,0.000000,0.030303,0.042301,0.227157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213139,560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.372684,NaN,...,1.4,58.149483,0.418505,241.80229,0.301663,0.324047,0.366569,0.308305,0.094029,0.000000
213140,560459511002,0.632111,NaN,NaN,NaN,NaN,NaN,0.632111,NaN,NaN,...,1.4,59.694490,0.403055,240.25730,0.288010,0.258713,0.825737,0.197500,0.062579,0.014547
213141,560459513001,0.868810,NaN,0.868810,NaN,0.868810,0.868810,NaN,NaN,1.737619,...,91.0,66.851970,0.331480,233.09980,0.232123,0.330784,0.105163,0.556757,0.261512,0.000000
213142,560459513002,NaN,NaN,0.970874,NaN,NaN,0.970874,NaN,NaN,NaN,...,91.0,64.308490,0.356915,235.64330,0.456522,0.457810,0.244165,0.595122,0.098058,0.000000


In [25]:
out.columns

Index(['GEOID', 'fires_per_1K_pop_2009', 'fires_per_1K_pop_2010',
       'fires_per_1K_pop_2011', 'fires_per_1K_pop_2012',
       'fires_per_1K_pop_2013', 'fires_per_1K_pop_2014',
       'fires_per_1K_pop_2015', 'fires_per_1K_pop_2016',
       'fires_per_1K_pop_2017', 'fire_propensity_risk_score',
       'fire_severity_risk_score', 'recorded_home_visits',
       'detectors_working_prc', 'detectors_working_CI', 'geography',
       'Population Density (per square mile), 2010', 'detectors_predicted',
       'smoke_detector_risk_score', 'CombinedRiskScore',
       'did_not_work_past_12_mo', 'house_pct_live_alone', 'pct_alt_heat',
       'house_yr_pct_before_1960', 'inc_pct_poverty',
       'race_pct_black_or_amind'],
      dtype='object')

In [26]:
SVI =  DataLoaders.SVIData(ACS)
SVI.data


,inc_pct_poverty,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4
geos,,,,,
010010201001,0.191946,0.3913,0.5597,0.3735,0.0993
010010201002,0.044269,0.3913,0.5597,0.3735,0.0993
010010202001,0.382199,0.7065,0.3585,0.5821,0.7299
010010202002,0.111426,0.7065,0.3585,0.5821,0.7299
010010203001,0.042301,0.5659,0.8729,0.6429,0.3882
...,...,...,...,...,...
560459511001,0.094029,0.3925,0.6370,0.2331,0.8555
560459511002,0.062579,0.3925,0.6370,0.2331,0.8555
560459513001,0.261512,0.5093,0.6043,0.2862,0.6949


In [27]:
out = out.merge(SVI.data, how = 'left', left_on = 'GEOID', right_index = True, suffixes = ('', 'svi'))
out

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,...,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4
0,010010201001,1.342282,2.684564,NaN,NaN,NaN,NaN,NaN,1.342282,1.342282,...,0.221831,0.017606,0.221831,0.191946,0.214765,0.191946,0.3913,0.5597,0.3735,0.0993
1,010010201002,1.581028,0.790514,NaN,1.581028,0.790514,NaN,1.581028,0.790514,NaN,...,0.127193,0.032895,0.079229,0.044269,0.082213,0.044269,0.3913,0.5597,0.3735,0.0993
2,010010202001,1.047120,NaN,3.141361,NaN,1.047120,1.047120,1.047120,1.047120,NaN,...,0.344560,0.000000,0.135678,0.382199,0.591667,0.382199,0.7065,0.3585,0.5821,0.7299
3,010010202002,2.832861,2.832861,2.832861,0.944287,0.944287,NaN,0.944287,NaN,3.777148,...,0.338496,0.019912,0.235772,0.111426,0.461974,0.111426,0.7065,0.3585,0.5821,0.7299
4,010010203001,0.846024,1.269036,0.846024,0.423012,1.269036,0.423012,0.423012,0.846024,NaN,...,0.276699,0.000000,0.030303,0.042301,0.227157,0.042301,0.5659,0.8729,0.6429,0.3882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213139,560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.372684,NaN,...,0.324047,0.366569,0.308305,0.094029,0.000000,0.094029,0.3925,0.6370,0.2331,0.8555
213140,560459511002,0.632111,NaN,NaN,NaN,NaN,NaN,0.632111,NaN,NaN,...,0.258713,0.825737,0.197500,0.062579,0.014547,0.062579,0.3925,0.6370,0.2331,0.8555
213141,560459513001,0.868810,NaN,0.868810,NaN,0.868810,0.868810,NaN,NaN,1.737619,...,0.330784,0.105163,0.556757,0.261512,0.000000,0.261512,0.5093,0.6043,0.2862,0.6949
213142,560459513002,NaN,NaN,0.970874,NaN,NaN,0.970874,NaN,NaN,NaN,...,0.457810,0.244165,0.595122,0.098058,0.000000,0.098058,0.5093,0.6043,0.2862,0.6949


In [28]:
#get number of years with any recorded fires
years = ['fires_per_1K_pop_2009']
years_YN = ['fires_per_1K_pop_2009_YN']
for i in range (8):
    years.append('fires_per_1K_pop_'+str(2010+i))
    years_YN.append('fires_per_1K_pop_'+str(2010+i)+'_YN')
years

out['years_fire_records'] = 0
for y in years:
    #out[[y+'_YN']]  = 0
    out[[y+'_YN']] = np.where(out[[y]] >0, 1, 0)
    #out['years_fire_records'] = out['years_fire_records']+ out[[y+'_YN']]
out['years_fire_records'] = out.loc[:,years_YN ].sum(axis=1)
out = out.drop(years_YN, axis = 1)
out

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,...,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,years_fire_records
0,010010201001,1.342282,2.684564,NaN,NaN,NaN,NaN,NaN,1.342282,1.342282,...,0.017606,0.221831,0.191946,0.214765,0.191946,0.3913,0.5597,0.3735,0.0993,4
1,010010201002,1.581028,0.790514,NaN,1.581028,0.790514,NaN,1.581028,0.790514,NaN,...,0.032895,0.079229,0.044269,0.082213,0.044269,0.3913,0.5597,0.3735,0.0993,6
2,010010202001,1.047120,NaN,3.141361,NaN,1.047120,1.047120,1.047120,1.047120,NaN,...,0.000000,0.135678,0.382199,0.591667,0.382199,0.7065,0.3585,0.5821,0.7299,6
3,010010202002,2.832861,2.832861,2.832861,0.944287,0.944287,NaN,0.944287,NaN,3.777148,...,0.019912,0.235772,0.111426,0.461974,0.111426,0.7065,0.3585,0.5821,0.7299,7
4,010010203001,0.846024,1.269036,0.846024,0.423012,1.269036,0.423012,0.423012,0.846024,NaN,...,0.000000,0.030303,0.042301,0.227157,0.042301,0.5659,0.8729,0.6429,0.3882,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213139,560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.372684,NaN,...,0.366569,0.308305,0.094029,0.000000,0.094029,0.3925,0.6370,0.2331,0.8555,1
213140,560459511002,0.632111,NaN,NaN,NaN,NaN,NaN,0.632111,NaN,NaN,...,0.825737,0.197500,0.062579,0.014547,0.062579,0.3925,0.6370,0.2331,0.8555,2
213141,560459513001,0.868810,NaN,0.868810,NaN,0.868810,0.868810,NaN,NaN,1.737619,...,0.105163,0.556757,0.261512,0.000000,0.261512,0.5093,0.6043,0.2862,0.6949,5
213142,560459513002,NaN,NaN,0.970874,NaN,NaN,0.970874,NaN,NaN,NaN,...,0.244165,0.595122,0.098058,0.000000,0.098058,0.5093,0.6043,0.2862,0.6949,2


In [29]:
out = out.round(3)
out

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,...,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,years_fire_records
0,010010201001,1.342,2.685,NaN,NaN,NaN,NaN,NaN,1.342,1.342,...,0.018,0.222,0.192,0.215,0.192,0.391,0.560,0.374,0.099,4
1,010010201002,1.581,0.791,NaN,1.581,0.791,NaN,1.581,0.791,NaN,...,0.033,0.079,0.044,0.082,0.044,0.391,0.560,0.374,0.099,6
2,010010202001,1.047,NaN,3.141,NaN,1.047,1.047,1.047,1.047,NaN,...,0.000,0.136,0.382,0.592,0.382,0.706,0.358,0.582,0.730,6
3,010010202002,2.833,2.833,2.833,0.944,0.944,NaN,0.944,NaN,3.777,...,0.020,0.236,0.111,0.462,0.111,0.706,0.358,0.582,0.730,7
4,010010203001,0.846,1.269,0.846,0.423,1.269,0.423,0.423,0.846,NaN,...,0.000,0.030,0.042,0.227,0.042,0.566,0.873,0.643,0.388,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213139,560459511001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373,NaN,...,0.367,0.308,0.094,0.000,0.094,0.392,0.637,0.233,0.856,1
213140,560459511002,0.632,NaN,NaN,NaN,NaN,NaN,0.632,NaN,NaN,...,0.826,0.198,0.063,0.015,0.063,0.392,0.637,0.233,0.856,2
213141,560459513001,0.869,NaN,0.869,NaN,0.869,0.869,NaN,NaN,1.738,...,0.105,0.557,0.262,0.000,0.262,0.509,0.604,0.286,0.695,5
213142,560459513002,NaN,NaN,0.971,NaN,NaN,0.971,NaN,NaN,NaN,...,0.244,0.595,0.098,0.000,0.098,0.509,0.604,0.286,0.695,2


In [30]:
# ENTER STATE 2 number code here as string then run the following cells 
STATE_2_NUM_CODE = '13' #Georgia
#STATE_2_NUM_CODE = '24' #Maryland

In [31]:
p = Path.cwd()

shp_p =  p / 'Data' / 'Master Project Data'/ 'Tiger_censusBlocks_2016' / f'tl_2016_{STATE_2_NUM_CODE}_bg.zip'

gdf = geo.read_file( 'zip://' + str(shp_p) ) 

#out['GEOID'] = out.index

gdf = gdf.merge(out,on = 'GEOID')

In [32]:
gdf

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,years_fire_records
0,13,089,021202,2,130890212022,Block Group 2,G5030,S,2213448,79182,...,0.000,0.255,0.050,0.043,0.050,0.010,0.369,0.445,0.279,2
1,13,089,021208,2,130890212082,Block Group 2,G5030,S,1124763,0,...,0.032,0.064,0.062,0.065,0.062,0.316,0.066,0.757,0.486,3
2,13,021,013102,3,130210131023,Block Group 3,G5030,S,1000826,0,...,0.017,0.066,0.341,0.916,0.341,0.954,0.760,0.790,0.519,6
3,13,021,013102,4,130210131024,Block Group 4,G5030,S,807552,0,...,0.000,0.025,0.314,0.820,0.314,0.954,0.760,0.790,0.519,9
4,13,195,020500,1,131950205001,Block Group 1,G5030,S,18496707,121296,...,0.064,0.096,0.144,0.141,0.144,0.665,0.862,0.555,0.162,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5421,13,015,960700,4,130159607004,Block Group 4,G5030,S,971909,0,...,0.000,0.106,0.329,0.291,0.329,0.787,0.821,0.791,0.960,5
5422,13,095,000501,2,130950005012,Block Group 2,G5030,S,2133016,53939,...,0.000,0.046,0.130,0.313,0.130,0.338,0.106,0.377,0.336,8
5423,13,095,000502,3,130950005023,Block Group 3,G5030,S,1526788,7359,...,0.000,0.117,0.014,0.251,0.014,0.155,0.957,0.412,0.237,5
5424,13,317,010302,5,133170103025,Block Group 5,G5030,S,1956404,0,...,0.026,0.405,0.195,0.583,0.195,0.718,0.980,0.665,0.679,7


In [33]:
#overview map- each model
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = ['GEOID', 'CombinedRiskScore','fire_propensity_risk_score','fire_severity_risk_score','smoke_detector_risk_score']

layers = ['CombinedRiskScore','fire_propensity_risk_score', 'fire_severity_risk_score', 'smoke_detector_risk_score' ]
for l in layers:
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)


m.save('georgia_overview.html')

In [34]:
#smoke alarm and svi
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

# plot chorpleth over the base map
themes = ['RPL_THEME1','RPL_THEME2','RPL_THEME3','RPL_THEME4']
cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', 'smoke_detector_risk_score'],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = 'smoke_detector_risk_score',
                  name = 'smoke_detector_risk_score'
                  ).add_to(m)    # name on the legend color bar
 
folium.GeoJsonTooltip(['GEOID', 'smoke_detector_risk_score', 'geography','detectors_predicted']+themes).add_to(cp.geojson)

for t in themes:
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', t],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = t,
                  name = t
                  ).add_to(m)    # name on the legend color bar
 
    folium.GeoJsonTooltip(['GEOID']+['smoke_detector_risk_score']+themes).add_to(cp.geojson)
# add layer controls
folium.LayerControl().add_to(m)


m.save('georgia_smoke_svi.html')

In [35]:
#inputs for propensity and severity models
start_loc = gdf.loc[1,'INTPTLAT':'INTPTLON'].to_list()

m = folium.Map(location=start_loc, zoom_start=11 , tiles=None)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='cartodbpositron').add_to(base_map)
base_map.add_to(m)

toolTip = {}
toolTip['fire_propensity_risk_score'] = ['GEOID', 'fire_propensity_risk_score']+ACS_cols_to_use+['years_fire_records']
toolTip['fire_severity_risk_score'] = ['GEOID', 'fire_severity_risk_score']+ACS_cols_to_use+['years_fire_records']

layers = ['fire_propensity_risk_score', 'fire_severity_risk_score']
for l in layers:
    # plot chorpleth over the base map
    cp = folium.Choropleth(gdf,                                # geo data
                  data=gdf,                           # data
                  key_on='feature.properties.GEOID', # feature.properties.key
                  columns=['GEOID', l],   # [key, value]
                  fill_color='RdPu',                     # cmap
                  line_weight=0.1,                       # line wight (of the border)
                  line_opacity=0.5,                      # line opacity (of the border)
                  overlay = False,
                  legend_name = l.replace('_',' '),
                  name =  l.replace('_',' ')
                  ).add_to(m)    # name on the legend color bar
    folium.GeoJsonTooltip(toolTip[l]).add_to(cp.geojson)

# add layer controls
folium.LayerControl().add_to(m)
#layer1.add_to(m)


m.save('georgia_fire_models.html')

In [44]:
pd.set_option("display.max_columns", None)

In [45]:
geoid1 = out[out['GEOID'].str[:-1]=='13089023603']
geoid1

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,fire_propensity_risk_score,fire_severity_risk_score,recorded_home_visits,detectors_working_prc,detectors_working_CI,geography,"Population Density (per square mile), 2010",detectors_predicted,smoke_detector_risk_score,CombinedRiskScore,did_not_work_past_12_mo,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,years_fire_records
53087,130890236031,1.402,2.803,4.205,2.102,0.701,2.803,0.701,NaN,0.701,0.383,0.371,0.0,50.0,5.041,county,3060.8,69.511,0.305,230.441,0.557,0.288,0.032,0.255,0.212,0.861,0.212,0.815,0.881,0.464,0.77,8
53088,130890236032,0.540,1.621,3.241,1.080,3.782,3.241,0.540,0.54,NaN,0.462,0.563,0.0,50.0,5.041,county,3060.8,70.296,0.297,229.655,0.429,0.404,0.023,0.289,0.311,0.907,0.311,0.815,0.881,0.464,0.77,8


In [46]:
geoid2 = out[out['GEOID']=='130890235043']
geoid2

,GEOID,fires_per_1K_pop_2009,fires_per_1K_pop_2010,fires_per_1K_pop_2011,fires_per_1K_pop_2012,fires_per_1K_pop_2013,fires_per_1K_pop_2014,fires_per_1K_pop_2015,fires_per_1K_pop_2016,fires_per_1K_pop_2017,fire_propensity_risk_score,fire_severity_risk_score,recorded_home_visits,detectors_working_prc,detectors_working_CI,geography,"Population Density (per square mile), 2010",detectors_predicted,smoke_detector_risk_score,CombinedRiskScore,did_not_work_past_12_mo,house_pct_live_alone,pct_alt_heat,house_yr_pct_before_1960,inc_pct_poverty,race_pct_black_or_amind,inc_pct_povertysvi,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,years_fire_records
53073,130890235043,0.657,0.657,1.314,1.314,0.657,0.657,1.314,0.657,1.314,0.941,0.497,0.0,50.0,5.041,county,4124.0,72.428,0.276,227.524,0.495,0.395,0.0,0.245,0.484,0.947,0.484,0.916,0.987,0.476,0.674,9


In [ ]:
for i in range(134):
    print(gdf.columns[i])